In [ ]:
# PSO_regression_task1.py
# Task 1: LLM controls c1, c2 + provides particle position/velocity history

import os
import time
import json
import math
import random
import requests
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

# ==========================================
# 1. CONFIGURATION
# ==========================================
class Config:
    AVALAI_API_KEY = ""
    AVALAI_BASE_URL = "https://api.avalai.ir/v1/chat/completions"
    LLM_MODEL = "gpt-5.1"

    DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv"
    SEQ_LENGTH = 24
    TRAIN_SPLIT = 0.8
    BATCH_SIZE = 128

    POPULATION_SIZE = 4
    MAX_ITERATIONS = 10
    
    # Initial C1, C2 (will be updated by LLM)
    C1_INIT = 2.0
    C2_INIT = 2.0
    
    W_MAX = 0.9
    W_MIN = 0.4

    BOUNDS = {
        'layers_min': 1, 'layers_max': 6,
        'neurons_min': 16, 'neurons_max': 256
    }

    EPOCHS_EVAL = 2
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. ENHANCED MEMORY MODULE (with Position/Velocity History)
# ==========================================
class OptimizationMemory:
    def __init__(self):
        self.history = []  # Fitness history
        self.particle_states = []  # Position/Velocity snapshots
        self.c1_c2_history = []  # Track c1, c2 changes
    
    def add_record(self, layers, neurons, rmse):
        self.history.append({
            'params': [int(layers), int(neurons)],
            'rmse': float(rmse)
        })
    
    def add_particle_state(self, iteration, positions, velocities, fitness):
        """Store complete swarm state for LLM context"""
        state = {
            'iteration': iteration,
            'particles': []
        }
        for i in range(len(positions)):
            state['particles'].append({
                'id': i,
                'position': [round(positions[i][0], 2), round(positions[i][1], 2)],
                'velocity': [round(velocities[i][0], 3), round(velocities[i][1], 3)],
                'fitness': round(fitness[i], 5)
            })
        self.particle_states.append(state)
        # Keep only last 3 states to avoid token overflow
        if len(self.particle_states) > 3:
            self.particle_states.pop(0)
    
    def add_c1_c2_record(self, c1, c2, iteration):
        self.c1_c2_history.append({'iter': iteration, 'c1': c1, 'c2': c2})
    
    def get_context_string(self):
        if not self.history:
            return "No history yet."
        
        sorted_hist = sorted(self.history, key=lambda x: x['rmse'])
        best_3 = sorted_hist[:3]
        worst_3 = sorted_hist[-3:]
        
        context = "=== OPTIMIZATION MEMORY ===\n"
        context += "TOP 3 Configurations (Lowest RMSE):\n"
        for i, h in enumerate(best_3):
            context += f"  {i+1}. [L={h['params'][0]}, N={h['params'][1]}] -> RMSE: {h['rmse']:.5f}\n"
        
        context += "\nWORST 3 Configurations:\n"
        for i, h in enumerate(worst_3):
            context += f"  {i+1}. [L={h['params'][0]}, N={h['params'][1]}] -> RMSE: {h['rmse']:.5f}\n"
        
        return context
    
    def get_particle_state_string(self):
        """Return recent particle positions and velocities for LLM"""
        if not self.particle_states:
            return "No particle state history."
        
        context = "\n=== PARTICLE STATE HISTORY (Last 3 Iterations) ===\n"
        for state in self.particle_states:
            context += f"\n--- Iteration {state['iteration']} ---\n"
            for p in state['particles']:
                context += (f"  P{p['id']}: Pos=[L:{p['position'][0]:.1f}, N:{p['position'][1]:.0f}] "
                           f"Vel=[{p['velocity'][0]:+.2f}, {p['velocity'][1]:+.2f}] "
                           f"RMSE:{p['fitness']:.5f}\n")
        return context
    
    def get_c1_c2_history_string(self):
        if not self.c1_c2_history:
            return "No c1/c2 history."
        context = "\n=== C1/C2 HISTORY ===\n"
        for h in self.c1_c2_history[-5:]:
            context += f"  Iter {h['iter']}: c1={h['c1']:.2f}, c2={h['c2']:.2f}\n"
        return context

# ==========================================
# 3. DATA LOADING
# ==========================================
def get_data():
    print(f"Downloading dataset...")
    try:
        df = pd.read_csv(Config.DATASET_URL)
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None, None

    cols = ['pm2.5', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']
    df = df[cols]
    df['pm2.5'] = df['pm2.5'].ffill().bfill()
    df = df.dropna()
    
    values = df.values.astype('float32')
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(values)
    
    X, y = [], []
    for i in range(len(scaled) - Config.SEQ_LENGTH):
        X.append(scaled[i:i+Config.SEQ_LENGTH, :])
        y.append(scaled[i+Config.SEQ_LENGTH, 0])
    
    X, y = np.array(X), np.array(y)
    split = int(len(X) * Config.TRAIN_SPLIT)
    
    train_loader = DataLoader(TensorDataset(torch.from_numpy(X[:split]), torch.from_numpy(y[:split])), 
                              batch_size=Config.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(TensorDataset(torch.from_numpy(X[split:]), torch.from_numpy(y[split:])), 
                             batch_size=Config.BATCH_SIZE)
    
    return train_loader, test_loader, X.shape[2], scaler

# ==========================================
# 4. MODEL
# ==========================================
class DynamicLSTM(nn.Module):
    def __init__(self, input_dim, num_layers, hidden_size):
        super().__init__()
        self.num_layers = int(num_layers)
        self.hidden_size = int(hidden_size)
        self.lstm = nn.LSTM(input_dim, self.hidden_size, self.num_layers, 
                           batch_first=True, dropout=0.2 if self.num_layers > 1 else 0)
        self.fc = nn.Linear(self.hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(Config.DEVICE)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(Config.DEVICE)
        out, _ = self.lstm(x, (h0, c0))
        return self.fc(out[:, -1, :])

# ==========================================
# 5. LLM INTERFACE (Enhanced: Returns c1, c2 + suggestions)
# ==========================================
def get_llm_suggestions_with_params(memory_context, particle_state_context, c1_c2_history, current_c1, current_c2):
    """
    LLM now returns:
    1. New hyperparameter suggestions
    2. Recommended c1, c2 values
    """
    prompt = f"""
You are an expert in PSO Hyperparameter Optimization for Deep Learning.

OBJECTIVE: Minimize RMSE for LSTM model.

SEARCH SPACE:
- Layers: {Config.BOUNDS['layers_min']} to {Config.BOUNDS['layers_max']}
- Neurons: {Config.BOUNDS['neurons_min']} to {Config.BOUNDS['neurons_max']}

CURRENT PSO PARAMETERS:
- c1 (cognitive): {current_c1:.2f}
- c2 (social): {current_c2:.2f}

{memory_context}

{particle_state_context}

{c1_c2_history}

=== YOUR TASKS ===

1. ANALYZE the particle velocities:
   - If velocities are too high (particles overshooting), suggest LOWER c1/c2.
   - If velocities are too low (stagnation), suggest HIGHER c1/c2.
   - Typical range: c1, c2 ∈ [1.0, 3.0]

2. SUGGEST 2 new configurations [layers, neurons] based on the history.

3. RECOMMEND new c1 and c2 values.

OUTPUT FORMAT (JSON only, no extra text):
{{
    "analysis": "Brief 1-sentence reasoning about velocity patterns...",
    "suggestions": [[layers1, neurons1], [layers2, neurons2]],
    "recommended_c1": float,
    "recommended_c2": float
}}
"""
    
    try:
        print("  [LLM] Consulting for suggestions + c1/c2 tuning...")
        resp = requests.post(
            Config.AVALAI_BASE_URL,
            headers={"Authorization": f"Bearer {Config.AVALAI_API_KEY}", "Content-Type": "application/json"},
            json={"model": Config.LLM_MODEL, "messages": [{"role": "user", "content": prompt}], "temperature": 0.5},
            timeout=30
        )
        
        if resp.status_code == 200:
            content = resp.json()['choices'][0]['message']['content']
            content = content.replace("```json", "").replace("```", "").strip()
            data = json.loads(content)
            
            print(f"  [LLM Analysis]: {data.get('analysis', 'N/A')}")
            
            suggestions = data.get("suggestions", [])
            new_c1 = float(data.get("recommended_c1", current_c1))
            new_c2 = float(data.get("recommended_c2", current_c2))
            
            # Clamp c1, c2 to valid range
            new_c1 = np.clip(new_c1, 1.0, 3.0)
            new_c2 = np.clip(new_c2, 1.0, 3.0)
            
            return suggestions, new_c1, new_c2
        else:
            print(f"  [LLM Error] Status: {resp.status_code}")
            return [], current_c1, current_c2
    except Exception as e:
        print(f"  [LLM Failed]: {e}")
        return [], current_c1, current_c2

# ==========================================
# 6. PSO WITH LLM-CONTROLLED C1/C2
# ==========================================
class LLMControlledPSO:
    def __init__(self, train_loader, test_loader, input_dim):
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.input_dim = input_dim
        self.memory = OptimizationMemory()
        
        # Initialize swarm
        self.positions = np.zeros((Config.POPULATION_SIZE, 2))
        self.positions[:, 0] = np.random.uniform(Config.BOUNDS['layers_min'], Config.BOUNDS['layers_max'], Config.POPULATION_SIZE)
        self.positions[:, 1] = np.random.uniform(Config.BOUNDS['neurons_min'], Config.BOUNDS['neurons_max'], Config.POPULATION_SIZE)
        
        self.velocities = np.zeros((Config.POPULATION_SIZE, 2))
        self.pbest_pos = self.positions.copy()
        self.pbest_val = np.full(Config.POPULATION_SIZE, float('inf'))
        self.gbest_pos = np.zeros(2)
        self.gbest_val = float('inf')
        
        # Dynamic c1, c2 (LLM will update these)
        self.c1 = Config.C1_INIT
        self.c2 = Config.C2_INIT

    def evaluate_particle(self, position):
        n_layers = int(np.clip(np.round(position[0]), Config.BOUNDS['layers_min'], Config.BOUNDS['layers_max']))
        n_neurons = int(np.clip(np.round(position[1]), Config.BOUNDS['neurons_min'], Config.BOUNDS['neurons_max']))
        
        print(f"    > Eval: L={n_layers}, N={n_neurons}...", end=" ")
        
        model = DynamicLSTM(self.input_dim, n_layers, n_neurons).to(Config.DEVICE)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        model.train()
        for _ in range(Config.EPOCHS_EVAL):
            for X_b, y_b in self.train_loader:
                X_b, y_b = X_b.to(Config.DEVICE), y_b.to(Config.DEVICE)
                optimizer.zero_grad()
                loss = criterion(model(X_b).squeeze(), y_b)
                loss.backward()
                optimizer.step()
        
        model.eval()
        total_mse, count = 0, 0
        with torch.no_grad():
            for X_b, y_b in self.test_loader:
                X_b, y_b = X_b.to(Config.DEVICE), y_b.to(Config.DEVICE)
                total_mse += criterion(model(X_b).squeeze(), y_b).item() * X_b.size(0)
                count += X_b.size(0)
        
        rmse = math.sqrt(total_mse / count)
        print(f"RMSE: {rmse:.5f}")
        
        self.memory.add_record(n_layers, n_neurons, rmse)
        return rmse

    def run(self):
        print("\n=== Starting LLM-Controlled PSO (c1/c2 + Particle History) ===")
        
        # Initial evaluation
        fitness = np.array([self.evaluate_particle(p) for p in self.positions])
        self.pbest_val = fitness.copy()
        best_idx = fitness.argmin()
        self.gbest_val = fitness[best_idx]
        self.gbest_pos = self.positions[best_idx].copy()
        
        for iteration in range(Config.MAX_ITERATIONS):
            # Adaptive inertia
            w = Config.W_MAX - ((Config.W_MAX - Config.W_MIN) * iteration / Config.MAX_ITERATIONS)
            
            print(f"\nIter {iteration+1}/{Config.MAX_ITERATIONS} | Best RMSE: {self.gbest_val:.5f} | c1={self.c1:.2f}, c2={self.c2:.2f}")
            
            # Store particle state for LLM
            self.memory.add_particle_state(iteration, self.positions, self.velocities, fitness)
            
            # LLM intervention at specific iterations
            if iteration in [2, 5, 8]:
                mem_ctx = self.memory.get_context_string()
                particle_ctx = self.memory.get_particle_state_string()
                c1c2_ctx = self.memory.get_c1_c2_history_string()
                
                suggestions, new_c1, new_c2 = get_llm_suggestions_with_params(
                    mem_ctx, particle_ctx, c1c2_ctx, self.c1, self.c2
                )
                
                # Update c1, c2 from LLM
                if new_c1 != self.c1 or new_c2 != self.c2:
                    print(f"  [LLM] Updating: c1={self.c1:.2f}->{new_c1:.2f}, c2={self.c2:.2f}->{new_c2:.2f}")
                    self.c1, self.c2 = new_c1, new_c2
                    self.memory.add_c1_c2_record(self.c1, self.c2, iteration)
                
                # Apply suggestions
                if suggestions:
                    worst_indices = np.argsort(fitness)[-len(suggestions):]
                    for idx, sug in zip(worst_indices, suggestions):
                        print(f"  [LLM] Replacing P{idx} -> {sug}")
                        self.positions[idx] = np.array(sug)
                        self.velocities[idx] = np.zeros(2)
                        new_fit = self.evaluate_particle(self.positions[idx])
                        fitness[idx] = new_fit
                        if new_fit < self.gbest_val:
                            self.gbest_val = new_fit
                            self.gbest_pos = self.positions[idx].copy()
                            print(f"  *** LLM FOUND NEW GLOBAL BEST! ***")
            
            # PSO update
            for i in range(Config.POPULATION_SIZE):
                r1, r2 = np.random.rand(2), np.random.rand(2)
                
                # Cognitive-only for last particle
                social_term = 0 if i == Config.POPULATION_SIZE - 1 else self.c2 * r2 * (self.gbest_pos - self.positions[i])
                cognitive_term = self.c1 * r1 * (self.pbest_pos[i] - self.positions[i])
                
                self.velocities[i] = w * self.velocities[i] + cognitive_term + social_term
                self.positions[i] += self.velocities[i]
                
                # Bounds
                self.positions[i, 0] = np.clip(self.positions[i, 0], Config.BOUNDS['layers_min'], Config.BOUNDS['layers_max'])
                self.positions[i, 1] = np.clip(self.positions[i, 1], Config.BOUNDS['neurons_min'], Config.BOUNDS['neurons_max'])
                
                fit = self.evaluate_particle(self.positions[i])
                fitness[i] = fit
                
                if fit < self.pbest_val[i]:
                    self.pbest_val[i] = fit
                    self.pbest_pos[i] = self.positions[i].copy()
                if fit < self.gbest_val:
                    self.gbest_val = fit
                    self.gbest_pos = self.positions[i].copy()
        
        return self.gbest_pos, self.gbest_val

# ==========================================
# 7. MAIN
# ==========================================
def main():
    print("=" * 70)
    print("TASK 1: LLM Controls c1/c2 + Particle Position/Velocity History")
    print("=" * 70)
    
    train_loader, test_loader, input_dim, _ = get_data()
    if train_loader is None:
        return
    
    start = time.time()
    pso = LLMControlledPSO(train_loader, test_loader, input_dim)
    best_pos, best_rmse = pso.run()
    elapsed = time.time() - start
    
    print("\n" + "=" * 70)
    print(f"RESULT: Best Params = [L={int(best_pos[0])}, N={int(best_pos[1])}]")
    print(f"        Best RMSE = {best_rmse:.5f}")
    print(f"        Time = {elapsed:.1f}s")
    print(f"        Final c1={pso.c1:.2f}, c2={pso.c2:.2f}")
    print("=" * 70)

if __name__ == "__main__":
    main()

TASK 1: LLM Controls c1/c2 + Particle Position/Velocity History

=== Starting LLM-Controlled PSO (c1/c2 + Particle History) ===
    > Eval: L=1, N=150... RMSE: 0.02816
    > Eval: L=1, N=179... RMSE: 0.02312
    > Eval: L=5, N=16... RMSE: 0.03346
    > Eval: L=2, N=202... RMSE: 0.02222

Iter 1/10 | Best RMSE: 0.02222 | c1=2.00, c2=2.00
    > Eval: L=1, N=213... RMSE: 0.02355
    > Eval: L=2, N=202... RMSE: 0.02161
    > Eval: L=3, N=256... RMSE: 0.02203
    > Eval: L=2, N=202... RMSE: 0.02593

Iter 2/10 | Best RMSE: 0.02161 | c1=2.00, c2=2.00
    > Eval: L=2, N=256... RMSE: 0.02303
    > Eval: L=2, N=222... RMSE: 0.02169
    > Eval: L=1, N=256... RMSE: 0.02289
    > Eval: L=2, N=202... RMSE: 0.02150

Iter 3/10 | Best RMSE: 0.02150 | c1=2.00, c2=2.00
  [LLM] Consulting for suggestions + c1/c2 tuning...
  [LLM Analysis]: Velocities, especially in neuron dimension, are large and cause boundary hits and oscillation around the best region, indicating overshooting rather than stagnation.
  [

In [ ]:
# PSO_regression_task2.py
# Task 2: Expand search space to include learning_rate, dropout, epochs

import os
import time
import json
import math
import random
import requests
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

# ==========================================
# 1. CONFIGURATION (EXPANDED SEARCH SPACE)
# ==========================================
class Config:
    AVALAI_API_KEY = ""
    AVALAI_BASE_URL = "https://api.avalai.ir/v1/chat/completions"
    LLM_MODEL = "gpt-5.1"

    DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv"
    SEQ_LENGTH = 24
    TRAIN_SPLIT = 0.8
    BATCH_SIZE = 128

    POPULATION_SIZE = 5
    MAX_ITERATIONS = 10
    C1 = 2.0
    C2 = 2.0
    W_MAX = 0.9
    W_MIN = 0.4

    # EXPANDED SEARCH SPACE (5 dimensions now!)
    BOUNDS = {
        'layers_min': 1, 'layers_max': 6,
        'neurons_min': 16, 'neurons_max': 256,
        'lr_min': 0.0001, 'lr_max': 0.01,        # Learning rate
        'dropout_min': 0.0, 'dropout_max': 0.5,   # Dropout rate
        'epochs_min': 1, 'epochs_max': 5          # Training epochs
    }
    
    # Dimension indices
    DIM_LAYERS = 0
    DIM_NEURONS = 1
    DIM_LR = 2
    DIM_DROPOUT = 3
    DIM_EPOCHS = 4
    NUM_DIMS = 5

    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. MEMORY MODULE
# ==========================================
class OptimizationMemory:
    def __init__(self):
        self.history = []
    
    def add_record(self, params, rmse):
        """params = [layers, neurons, lr, dropout, epochs]"""
        self.history.append({'params': params, 'rmse': float(rmse)})
    
    def get_context_string(self):
        if not self.history:
            return "No history yet."
        
        sorted_hist = sorted(self.history, key=lambda x: x['rmse'])
        best_5 = sorted_hist[:5]
        worst_3 = sorted_hist[-3:]
        
        context = "=== OPTIMIZATION MEMORY (5D Search Space) ===\n"
        context += "TOP 5 Configurations:\n"
        for i, h in enumerate(best_5):
            p = h['params']
            context += f"  {i+1}. [L={p[0]}, N={p[1]}, LR={p[2]:.4f}, Drop={p[3]:.2f}, Ep={p[4]}] -> RMSE: {h['rmse']:.5f}\n"
        
        context += "\nWORST 3 Configurations:\n"
        for i, h in enumerate(worst_3):
            p = h['params']
            context += f"  {i+1}. [L={p[0]}, N={p[1]}, LR={p[2]:.4f}, Drop={p[3]:.2f}, Ep={p[4]}] -> RMSE: {h['rmse']:.5f}\n"
        
        return context

# ==========================================
# 3. DATA LOADING
# ==========================================
def get_data():
    print("Downloading dataset...")
    try:
        df = pd.read_csv(Config.DATASET_URL)
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None, None

    cols = ['pm2.5', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']
    df = df[cols]
    df['pm2.5'] = df['pm2.5'].ffill().bfill()
    df = df.dropna()
    
    values = df.values.astype('float32')
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(values)
    
    X, y = [], []
    for i in range(len(scaled) - Config.SEQ_LENGTH):
        X.append(scaled[i:i+Config.SEQ_LENGTH, :])
        y.append(scaled[i+Config.SEQ_LENGTH, 0])
    
    X, y = np.array(X), np.array(y)
    split = int(len(X) * Config.TRAIN_SPLIT)
    
    train_loader = DataLoader(TensorDataset(torch.from_numpy(X[:split]), torch.from_numpy(y[:split])), 
                              batch_size=Config.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(TensorDataset(torch.from_numpy(X[split:]), torch.from_numpy(y[split:])), 
                             batch_size=Config.BATCH_SIZE)
    
    return train_loader, test_loader, X.shape[2], scaler

# ==========================================
# 4. MODEL (with configurable dropout)
# ==========================================
class DynamicLSTM(nn.Module):
    def __init__(self, input_dim, num_layers, hidden_size, dropout_rate):
        super().__init__()
        self.num_layers = int(num_layers)
        self.hidden_size = int(hidden_size)
        self.dropout_rate = float(dropout_rate)
        
        self.lstm = nn.LSTM(
            input_dim, self.hidden_size, self.num_layers,
            batch_first=True,
            dropout=self.dropout_rate if self.num_layers > 1 else 0
        )
        self.dropout = nn.Dropout(self.dropout_rate)
        self.fc = nn.Linear(self.hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(Config.DEVICE)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(Config.DEVICE)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        return self.fc(out)

# ==========================================
# 5. LLM INTERFACE (5D suggestions)
# ==========================================
def get_llm_suggestions_5d(memory_context):
    prompt = f"""
You are an expert in PSO Hyperparameter Optimization for LSTM models.

OBJECTIVE: Minimize RMSE.

SEARCH SPACE (5 Dimensions):
1. Layers: {Config.BOUNDS['layers_min']} to {Config.BOUNDS['layers_max']} (integer)
2. Neurons: {Config.BOUNDS['neurons_min']} to {Config.BOUNDS['neurons_max']} (integer)
3. Learning Rate: {Config.BOUNDS['lr_min']} to {Config.BOUNDS['lr_max']} (float)
4. Dropout: {Config.BOUNDS['dropout_min']} to {Config.BOUNDS['dropout_max']} (float)
5. Epochs: {Config.BOUNDS['epochs_min']} to {Config.BOUNDS['epochs_max']} (integer)

{memory_context}

TASK: Suggest 2 new configurations based on the patterns you observe.
- Consider interactions between parameters (e.g., more layers may need lower LR).
- Balance exploration and exploitation.

OUTPUT FORMAT (JSON only):
{{
    "analysis": "Brief reasoning...",
    "suggestions": [
        [layers1, neurons1, lr1, dropout1, epochs1],
        [layers2, neurons2, lr2, dropout2, epochs2]
    ]
}}
"""
    
    try:
        print("  [LLM] Consulting for 5D suggestions...")
        resp = requests.post(
            Config.AVALAI_BASE_URL,
            headers={"Authorization": f"Bearer {Config.AVALAI_API_KEY}", "Content-Type": "application/json"},
            json={"model": Config.LLM_MODEL, "messages": [{"role": "user", "content": prompt}], "temperature": 0.6},
            timeout=30
        )
        
        if resp.status_code == 200:
            content = resp.json()['choices'][0]['message']['content']
            content = content.replace("```json", "").replace("```", "").strip()
            data = json.loads(content)
            print(f"  [LLM Analysis]: {data.get('analysis', 'N/A')}")
            return data.get("suggestions", [])
        else:
            print(f"  [LLM Error] Status: {resp.status_code}")
            return []
    except Exception as e:
        print(f"  [LLM Failed]: {e}")
        return []

# ==========================================
# 6. 5D PSO
# ==========================================
class ExpandedPSO:
    def __init__(self, train_loader, test_loader, input_dim, use_llm=True):
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.input_dim = input_dim
        self.use_llm = use_llm
        self.memory = OptimizationMemory()
        
        # Initialize 5D swarm
        self.positions = np.zeros((Config.POPULATION_SIZE, Config.NUM_DIMS))
        self.positions[:, Config.DIM_LAYERS] = np.random.uniform(Config.BOUNDS['layers_min'], Config.BOUNDS['layers_max'], Config.POPULATION_SIZE)
        self.positions[:, Config.DIM_NEURONS] = np.random.uniform(Config.BOUNDS['neurons_min'], Config.BOUNDS['neurons_max'], Config.POPULATION_SIZE)
        self.positions[:, Config.DIM_LR] = np.random.uniform(Config.BOUNDS['lr_min'], Config.BOUNDS['lr_max'], Config.POPULATION_SIZE)
        self.positions[:, Config.DIM_DROPOUT] = np.random.uniform(Config.BOUNDS['dropout_min'], Config.BOUNDS['dropout_max'], Config.POPULATION_SIZE)
        self.positions[:, Config.DIM_EPOCHS] = np.random.uniform(Config.BOUNDS['epochs_min'], Config.BOUNDS['epochs_max'], Config.POPULATION_SIZE)
        
        self.velocities = np.zeros((Config.POPULATION_SIZE, Config.NUM_DIMS))
        self.pbest_pos = self.positions.copy()
        self.pbest_val = np.full(Config.POPULATION_SIZE, float('inf'))
        self.gbest_pos = np.zeros(Config.NUM_DIMS)
        self.gbest_val = float('inf')

    def clip_position(self, pos):
        """Clip all dimensions to valid ranges"""
        pos[Config.DIM_LAYERS] = np.clip(pos[Config.DIM_LAYERS], Config.BOUNDS['layers_min'], Config.BOUNDS['layers_max'])
        pos[Config.DIM_NEURONS] = np.clip(pos[Config.DIM_NEURONS], Config.BOUNDS['neurons_min'], Config.BOUNDS['neurons_max'])
        pos[Config.DIM_LR] = np.clip(pos[Config.DIM_LR], Config.BOUNDS['lr_min'], Config.BOUNDS['lr_max'])
        pos[Config.DIM_DROPOUT] = np.clip(pos[Config.DIM_DROPOUT], Config.BOUNDS['dropout_min'], Config.BOUNDS['dropout_max'])
        pos[Config.DIM_EPOCHS] = np.clip(pos[Config.DIM_EPOCHS], Config.BOUNDS['epochs_min'], Config.BOUNDS['epochs_max'])
        return pos

    def decode_position(self, pos):
        """Convert continuous position to actual hyperparameters"""
        return {
            'layers': int(np.round(pos[Config.DIM_LAYERS])),
            'neurons': int(np.round(pos[Config.DIM_NEURONS])),
            'lr': float(pos[Config.DIM_LR]),
            'dropout': float(pos[Config.DIM_DROPOUT]),
            'epochs': int(np.round(pos[Config.DIM_EPOCHS]))
        }

    def evaluate_particle(self, position):
        params = self.decode_position(position)
        
        print(f"    > Eval: L={params['layers']}, N={params['neurons']}, LR={params['lr']:.4f}, Drop={params['dropout']:.2f}, Ep={params['epochs']}...", end=" ")
        
        try:
            model = DynamicLSTM(self.input_dim, params['layers'], params['neurons'], params['dropout']).to(Config.DEVICE)
            criterion = nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=params['lr'])
            
            model.train()
            for _ in range(params['epochs']):
                for X_b, y_b in self.train_loader:
                    X_b, y_b = X_b.to(Config.DEVICE), y_b.to(Config.DEVICE)
                    optimizer.zero_grad()
                    loss = criterion(model(X_b).squeeze(), y_b)
                    loss.backward()
                    optimizer.step()
            
            model.eval()
            total_mse, count = 0, 0
            with torch.no_grad():
                for X_b, y_b in self.test_loader:
                    X_b, y_b = X_b.to(Config.DEVICE), y_b.to(Config.DEVICE)
                    total_mse += criterion(model(X_b).squeeze(), y_b).item() * X_b.size(0)
                    count += X_b.size(0)
            
            rmse = math.sqrt(total_mse / count)
            print(f"RMSE: {rmse:.5f}")
            
            self.memory.add_record([params['layers'], params['neurons'], params['lr'], params['dropout'], params['epochs']], rmse)
            return rmse
        except Exception as e:
            print(f"Error: {e}")
            return float('inf')

    def run(self):
        mode = "LLM-Enhanced" if self.use_llm else "Vanilla"
        print(f"\n=== Starting 5D {mode} PSO ===")
        
        # Initial evaluation
        fitness = np.array([self.evaluate_particle(p) for p in self.positions])
        self.pbest_val = fitness.copy()
        best_idx = fitness.argmin()
        self.gbest_val = fitness[best_idx]
        self.gbest_pos = self.positions[best_idx].copy()
        
        for iteration in range(Config.MAX_ITERATIONS):
            w = Config.W_MAX - ((Config.W_MAX - Config.W_MIN) * iteration / Config.MAX_ITERATIONS)
            print(f"\nIter {iteration+1}/{Config.MAX_ITERATIONS} | Best RMSE: {self.gbest_val:.5f}")
            
            # LLM intervention
            if self.use_llm and iteration in [2, 5, 8]:
                suggestions = get_llm_suggestions_5d(self.memory.get_context_string())
                if suggestions:
                    worst_indices = np.argsort(fitness)[-len(suggestions):]
                    for idx, sug in zip(worst_indices, suggestions):
                        print(f"  [LLM] Replacing P{idx} -> {sug}")
                        self.positions[idx] = np.array(sug)
                        self.velocities[idx] = np.zeros(Config.NUM_DIMS)
                        new_fit = self.evaluate_particle(self.positions[idx])
                        fitness[idx] = new_fit
                        if new_fit < self.gbest_val:
                            self.gbest_val = new_fit
                            self.gbest_pos = self.positions[idx].copy()
                            print(f"  *** LLM FOUND NEW GLOBAL BEST! ***")
            
            # PSO update
            for i in range(Config.POPULATION_SIZE):
                r1 = np.random.rand(Config.NUM_DIMS)
                r2 = np.random.rand(Config.NUM_DIMS)
                
                social_term = 0 if i == Config.POPULATION_SIZE - 1 else Config.C2 * r2 * (self.gbest_pos - self.positions[i])
                cognitive_term = Config.C1 * r1 * (self.pbest_pos[i] - self.positions[i])
                
                self.velocities[i] = w * self.velocities[i] + cognitive_term + social_term
                self.positions[i] += self.velocities[i]
                self.positions[i] = self.clip_position(self.positions[i])
                
                fit = self.evaluate_particle(self.positions[i])
                fitness[i] = fit
                
                if fit < self.pbest_val[i]:
                    self.pbest_val[i] = fit
                    self.pbest_pos[i] = self.positions[i].copy()
                if fit < self.gbest_val:
                    self.gbest_val = fit
                    self.gbest_pos = self.positions[i].copy()
        
        return self.gbest_pos, self.gbest_val

# ==========================================
# 7. MAIN
# ==========================================
def main():
    print("=" * 80)
    print("TASK 2: Expanded 5D Search Space (Layers, Neurons, LR, Dropout, Epochs)")
    print("=" * 80)
    
    train_loader, test_loader, input_dim, _ = get_data()
    if train_loader is None:
        return
    
    # Run Vanilla 5D PSO
    print("\n" + "-" * 40)
    print("Running VANILLA 5D PSO...")
    start = time.time()
    pso_v = ExpandedPSO(train_loader, test_loader, input_dim, use_llm=False)
    best_v, rmse_v = pso_v.run()
    time_v = time.time() - start
    
    # Run LLM-Enhanced 5D PSO
    print("\n" + "-" * 40)
    print("Running LLM-ENHANCED 5D PSO...")
    start = time.time()
    pso_l = ExpandedPSO(train_loader, test_loader, input_dim, use_llm=True)
    best_l, rmse_l = pso_l.run()
    time_l = time.time() - start
    
    # Results
    print("\n" + "=" * 80)
    print(f"{'METHOD':<20} | {'LAYERS':<6} | {'NEURONS':<7} | {'LR':<8} | {'DROP':<5} | {'EPOCHS':<6} | {'RMSE':<10} | {'TIME'}")
    print("-" * 80)
    
    v_params = pso_v.decode_position(best_v)
    l_params = pso_l.decode_position(best_l)
    
    print(f"{'Vanilla 5D PSO':<20} | {v_params['layers']:<6} | {v_params['neurons']:<7} | {v_params['lr']:<8.4f} | {v_params['dropout']:<5.2f} | {v_params['epochs']:<6} | {rmse_v:<10.5f} | {time_v:.1f}s")
    print(f"{'LLM-Enhanced 5D':<20} | {l_params['layers']:<6} | {l_params['neurons']:<7} | {l_params['lr']:<8.4f} | {l_params['dropout']:<5.2f} | {l_params['epochs']:<6} | {rmse_l:<10.5f} | {time_l:.1f}s")
    print("=" * 80)

if __name__ == "__main__":
    main()

TASK 2: Expanded 5D Search Space (Layers, Neurons, LR, Dropout, Epochs)

----------------------------------------
Running VANILLA 5D PSO...

=== Starting 5D Vanilla PSO ===
    > Eval: L=4, N=72, LR=0.0043, Drop=0.12, Ep=1... RMSE: 0.02218
    > Eval: L=2, N=35, LR=0.0031, Drop=0.47, Ep=1... RMSE: 0.02977
    > Eval: L=2, N=70, LR=0.0014, Drop=0.41, Ep=2... RMSE: 0.02708
    > Eval: L=5, N=219, LR=0.0054, Drop=0.08, Ep=4... RMSE: 0.02381
    > Eval: L=4, N=58, LR=0.0089, Drop=0.45, Ep=4... RMSE: 0.02341

Iter 1/10 | Best RMSE: 0.02218
    > Eval: L=4, N=72, LR=0.0043, Drop=0.12, Ep=1... RMSE: 0.02710
    > Eval: L=3, N=58, LR=0.0033, Drop=0.21, Ep=1... RMSE: 0.02801
    > Eval: L=2, N=71, LR=0.0041, Drop=0.04, Ep=2... RMSE: 0.02265
    > Eval: L=5, N=16, LR=0.0053, Drop=0.12, Ep=3... RMSE: 0.02357
    > Eval: L=4, N=58, LR=0.0089, Drop=0.45, Ep=4... RMSE: 0.02481

Iter 2/10 | Best RMSE: 0.02218
    > Eval: L=4, N=72, LR=0.0043, Drop=0.12, Ep=1... RMSE: 0.02320
    > Eval: L=5, N=102, L

In [ ]:
# PSO_regression_task3.py
# Task 3: Implement AIWPSO (Success Rate based W) + Rank-based W from paper

import os
import time
import json
import math
import random
import requests
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

# ==========================================
# 1. CONFIGURATION
# ==========================================
class Config:
    AVALAI_API_KEY = ""
    AVALAI_BASE_URL = "https://api.avalai.ir/v1/chat/completions"
    LLM_MODEL = "gpt-5.1"

    DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv"
    SEQ_LENGTH = 24
    TRAIN_SPLIT = 0.8
    BATCH_SIZE = 128

    POPULATION_SIZE = 5
    MAX_ITERATIONS = 10
    C1 = 2.0
    C2 = 2.0
    W_MAX = 1.0  # Paper uses [0, 1.0] range
    W_MIN = 0.0

    BOUNDS = {
        'layers_min': 1, 'layers_max': 6,
        'neurons_min': 16, 'neurons_max': 256
    }

    EPOCHS_EVAL = 2
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==========================================
# 2. DATA LOADING
# ==========================================
def get_data():
    print("Downloading dataset...")
    try:
        df = pd.read_csv(Config.DATASET_URL)
    except Exception as e:
        print(f"Error: {e}")
        return None, None, None, None

    cols = ['pm2.5', 'DEWP', 'TEMP', 'PRES', 'Iws', 'Is', 'Ir']
    df = df[cols]
    df['pm2.5'] = df['pm2.5'].ffill().bfill()
    df = df.dropna()
    
    values = df.values.astype('float32')
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(values)
    
    X, y = [], []
    for i in range(len(scaled) - Config.SEQ_LENGTH):
        X.append(scaled[i:i+Config.SEQ_LENGTH, :])
        y.append(scaled[i+Config.SEQ_LENGTH, 0])
    
    X, y = np.array(X), np.array(y)
    split = int(len(X) * Config.TRAIN_SPLIT)
    
    train_loader = DataLoader(TensorDataset(torch.from_numpy(X[:split]), torch.from_numpy(y[:split])), 
                              batch_size=Config.BATCH_SIZE, shuffle=True)
    test_loader = DataLoader(TensorDataset(torch.from_numpy(X[split:]), torch.from_numpy(y[split:])), 
                             batch_size=Config.BATCH_SIZE)
    
    return train_loader, test_loader, X.shape[2], scaler

# ==========================================
# 3. MODEL
# ==========================================
class DynamicLSTM(nn.Module):
    def __init__(self, input_dim, num_layers, hidden_size):
        super().__init__()
        self.num_layers = int(num_layers)
        self.hidden_size = int(hidden_size)
        self.lstm = nn.LSTM(input_dim, self.hidden_size, self.num_layers, 
                           batch_first=True, dropout=0.2 if self.num_layers > 1 else 0)
        self.fc = nn.Linear(self.hidden_size, 1)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(Config.DEVICE)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(Config.DEVICE)
        out, _ = self.lstm(x, (h0, c0))
        return self.fc(out[:, -1, :])

# ==========================================
# 4. INERTIA WEIGHT STRATEGIES (FROM PAPER)
# ==========================================
class InertiaWeightStrategy:
    """
    Implements various W strategies from the paper:
    "A novel particle swarm optimization algorithm with adaptive inertia weight"
    """
    
    @staticmethod
    def linear_decreasing(iteration, max_iter, w_max=0.9, w_min=0.4):
        """W3: Standard linear decrease"""
        return w_max - ((w_max - w_min) * iteration / max_iter)
    
    @staticmethod
    def linear_increasing(iteration, max_iter, w_max=0.9, w_min=0.4):
        """W9: Linear increase (opposite of standard)"""
        return w_min + ((w_max - w_min) * iteration / max_iter)
    
    @staticmethod
    def random_inertia():
        """W2: Random inertia weight"""
        return 0.5 + np.random.rand() / 2  # Range [0.5, 1.0]
    
    @staticmethod
    def success_rate_adaptive(success_rate, w_max=1.0, w_min=0.0):
        """
        AIWPSO (Proposed in paper - Equation 20):
        w(t) = (w_max - w_min) * Ps(t) + w_min
        
        High success rate -> High w (exploration)
        Low success rate -> Low w (exploitation)
        """
        return (w_max - w_min) * success_rate + w_min
    
    @staticmethod
    def rank_based(particle_rank, total_population, w_max=0.9, w_min=0.4):
        """
        W13: Rank-based inertia weight (Equation 16)
        Better particles get lower w (exploit), worse particles get higher w (explore)
        """
        return w_min + (w_max - w_min) * (particle_rank / total_population)

# ==========================================
# 5. PSO WITH MULTIPLE W STRATEGIES
# ==========================================
class AdaptivePSO:
    def __init__(self, train_loader, test_loader, input_dim, w_strategy='linear_decrease'):
        """
        w_strategy options:
        - 'linear_decrease': Standard decreasing (baseline)
        - 'linear_increase': Increasing W
        - 'random': Random W each iteration
        - 'success_rate': AIWPSO (paper's proposed method)
        - 'rank_based': Different W per particle based on rank
        """
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.input_dim = input_dim
        self.w_strategy = w_strategy
        
        # Initialize swarm
        self.positions = np.zeros((Config.POPULATION_SIZE, 2))
        self.positions[:, 0] = np.random.uniform(Config.BOUNDS['layers_min'], Config.BOUNDS['layers_max'], Config.POPULATION_SIZE)
        self.positions[:, 1] = np.random.uniform(Config.BOUNDS['neurons_min'], Config.BOUNDS['neurons_max'], Config.POPULATION_SIZE)
        
        self.velocities = np.zeros((Config.POPULATION_SIZE, 2))
        self.pbest_pos = self.positions.copy()
        self.pbest_val = np.full(Config.POPULATION_SIZE, float('inf'))
        self.gbest_pos = np.zeros(2)
        self.gbest_val = float('inf')
        
        # For success rate calculation
        self.prev_pbest_val = np.full(Config.POPULATION_SIZE, float('inf'))
        
        # Track W values for analysis
        self.w_history = []

    def evaluate_particle(self, position):
        n_layers = int(np.clip(np.round(position[0]), Config.BOUNDS['layers_min'], Config.BOUNDS['layers_max']))
        n_neurons = int(np.clip(np.round(position[1]), Config.BOUNDS['neurons_min'], Config.BOUNDS['neurons_max']))
        
        print(f"    > Eval: L={n_layers}, N={n_neurons}...", end=" ")
        
        model = DynamicLSTM(self.input_dim, n_layers, n_neurons).to(Config.DEVICE)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.001)
        
        model.train()
        for _ in range(Config.EPOCHS_EVAL):
            for X_b, y_b in self.train_loader:
                X_b, y_b = X_b.to(Config.DEVICE), y_b.to(Config.DEVICE)
                optimizer.zero_grad()
                loss = criterion(model(X_b).squeeze(), y_b)
                loss.backward()
                optimizer.step()
        
        model.eval()
        total_mse, count = 0, 0
        with torch.no_grad():
            for X_b, y_b in self.test_loader:
                X_b, y_b = X_b.to(Config.DEVICE), y_b.to(Config.DEVICE)
                total_mse += criterion(model(X_b).squeeze(), y_b).item() * X_b.size(0)
                count += X_b.size(0)
        
        rmse = math.sqrt(total_mse / count)
        print(f"RMSE: {rmse:.5f}")
        return rmse

    def calculate_success_rate(self):
        """
        Equation 17-18 from paper:
        Success = 1 if pbest improved, 0 otherwise
        Ps = sum(Success) / n
        """
        successes = 0
        for i in range(Config.POPULATION_SIZE):
            if self.pbest_val[i] < self.prev_pbest_val[i]:
                successes += 1
        return successes / Config.POPULATION_SIZE

    def get_inertia_weight(self, iteration, particle_idx=None, fitness=None):
        """Get W based on selected strategy"""
        if self.w_strategy == 'linear_decrease':
            return InertiaWeightStrategy.linear_decreasing(iteration, Config.MAX_ITERATIONS)
        
        elif self.w_strategy == 'linear_increase':
            return InertiaWeightStrategy.linear_increasing(iteration, Config.MAX_ITERATIONS)
        
        elif self.w_strategy == 'random':
            return InertiaWeightStrategy.random_inertia()
        
        elif self.w_strategy == 'success_rate':
            success_rate = self.calculate_success_rate()
            return InertiaWeightStrategy.success_rate_adaptive(success_rate)
        
        elif self.w_strategy == 'rank_based':
            if fitness is not None and particle_idx is not None:
                # Rank particles (0 = best, n-1 = worst)
                ranks = np.argsort(np.argsort(fitness))
                return InertiaWeightStrategy.rank_based(ranks[particle_idx], Config.POPULATION_SIZE)
            return 0.7  # Default
        
        else:
            return 0.7  # Default

    def run(self):
        print(f"\n=== Starting PSO with W Strategy: {self.w_strategy.upper()} ===")
        
        # Initial evaluation
        fitness = np.array([self.evaluate_particle(p) for p in self.positions])
        self.pbest_val = fitness.copy()
        self.prev_pbest_val = fitness.copy()
        best_idx = fitness.argmin()
        self.gbest_val = fitness[best_idx]
        self.gbest_pos = self.positions[best_idx].copy()
        
        for iteration in range(Config.MAX_ITERATIONS):
            # Store previous pbest for success rate calculation
            self.prev_pbest_val = self.pbest_val.copy()
            
            # Get base W (for strategies that use single W for all particles)
            base_w = self.get_inertia_weight(iteration, fitness=fitness)
            
            # Calculate success rate for logging
            if iteration > 0:
                success_rate = self.calculate_success_rate()
            else:
                success_rate = 1.0
            
            self.w_history.append(base_w)
            
            print(f"\nIter {iteration+1}/{Config.MAX_ITERATIONS} | Best RMSE: {self.gbest_val:.5f} | W={base_w:.3f} | Success Rate={success_rate:.2f}")
            
            # PSO update
            for i in range(Config.POPULATION_SIZE):
                # Get particle-specific W for rank-based strategy
                if self.w_strategy == 'rank_based':
                    w = self.get_inertia_weight(iteration, particle_idx=i, fitness=fitness)
                else:
                    w = base_w
                
                r1, r2 = np.random.rand(2), np.random.rand(2)
                
                # Cognitive-only for last particle
                social_term = 0 if i == Config.POPULATION_SIZE - 1 else Config.C2 * r2 * (self.gbest_pos - self.positions[i])
                cognitive_term = Config.C1 * r1 * (self.pbest_pos[i] - self.positions[i])
                
                self.velocities[i] = w * self.velocities[i] + cognitive_term + social_term
                self.positions[i] += self.velocities[i]
                
                # Bounds
                self.positions[i, 0] = np.clip(self.positions[i, 0], Config.BOUNDS['layers_min'], Config.BOUNDS['layers_max'])
                self.positions[i, 1] = np.clip(self.positions[i, 1], Config.BOUNDS['neurons_min'], Config.BOUNDS['neurons_max'])
                
                fit = self.evaluate_particle(self.positions[i])
                fitness[i] = fit
                
                if fit < self.pbest_val[i]:
                    self.pbest_val[i] = fit
                    self.pbest_pos[i] = self.positions[i].copy()
                if fit < self.gbest_val:
                    self.gbest_val = fit
                    self.gbest_pos = self.positions[i].copy()
        
        return self.gbest_pos, self.gbest_val, self.w_history

# ==========================================
# 6. MAIN - COMPARE ALL STRATEGIES
# ==========================================
def main():
    print("=" * 80)
    print("TASK 3: Compare Inertia Weight Strategies from Paper")
    print("Based on: 'A novel PSO algorithm with adaptive inertia weight'")
    print("=" * 80)
    
    train_loader, test_loader, input_dim, _ = get_data()
    if train_loader is None:
        return
    
    strategies = [
        'linear_decrease',   # W3: Standard baseline
        'linear_increase',   # W9: Opposite of standard
        'random',            # W2: Random W
        'success_rate',      # AIWPSO: Paper's proposed method
        'rank_based'         # W13: Per-particle W based on fitness rank
    ]
    
    results = []
    
    for strategy in strategies:
        print("\n" + "=" * 60)
        start = time.time()
        pso = AdaptivePSO(train_loader, test_loader, input_dim, w_strategy=strategy)
        best_pos, best_rmse, w_history = pso.run()
        elapsed = time.time() - start
        
        results.append({
            'strategy': strategy,
            'layers': int(np.round(best_pos[0])),
            'neurons': int(np.round(best_pos[1])),
            'rmse': best_rmse,
            'time': elapsed,
            'avg_w': np.mean(w_history),
            'w_std': np.std(w_history)
        })
    
    # Print comparison table
    print("\n" + "=" * 100)
    print("COMPARISON OF INERTIA WEIGHT STRATEGIES")
    print("=" * 100)
    print(f"{'STRATEGY':<20} | {'LAYERS':<6} | {'NEURONS':<7} | {'RMSE':<10} | {'TIME(s)':<8} | {'AVG W':<7} | {'W STD'}")
    print("-" * 100)
    
    for r in results:
        print(f"{r['strategy']:<20} | {r['layers']:<6} | {r['neurons']:<7} | {r['rmse']:<10.5f} | {r['time']:<8.1f} | {r['avg_w']:<7.3f} | {r['w_std']:.3f}")
    
    print("=" * 100)
    
    # Find best strategy
    best = min(results, key=lambda x: x['rmse'])
    print(f"\n🏆 BEST STRATEGY: {best['strategy'].upper()}")
    print(f"   Best RMSE: {best['rmse']:.5f}")
    print(f"   Parameters: L={best['layers']}, N={best['neurons']}")

if __name__ == "__main__":
    main()

TASK 3: Compare Inertia Weight Strategies from Paper
Based on: 'A novel PSO algorithm with adaptive inertia weight'


=== Starting PSO with W Strategy: LINEAR_DECREASE ===
    > Eval: L=4, N=238... RMSE: 0.02224
    > Eval: L=3, N=249... RMSE: 0.02391
    > Eval: L=4, N=117... RMSE: 0.02249
    > Eval: L=2, N=27... RMSE: 0.03350
    > Eval: L=1, N=46... RMSE: 0.02451

Iter 1/10 | Best RMSE: 0.02224 | W=0.900 | Success Rate=1.00
    > Eval: L=4, N=238... RMSE: 0.02156
    > Eval: L=4, N=241... RMSE: 0.02345
    > Eval: L=3, N=256... RMSE: 0.02299
    > Eval: L=5, N=234... RMSE: 0.02214
    > Eval: L=1, N=46... RMSE: 0.02916

Iter 2/10 | Best RMSE: 0.02156 | W=0.850 | Success Rate=0.00
    > Eval: L=4, N=238... RMSE: 0.02295
    > Eval: L=4, N=231... RMSE: 0.02190
    > Eval: L=5, N=256... RMSE: 0.02199
    > Eval: L=6, N=256... RMSE: 0.02190
    > Eval: L=1, N=46... RMSE: 0.02406

Iter 3/10 | Best RMSE: 0.02156 | W=0.800 | Success Rate=0.00
    > Eval: L=4, N=238... RMSE: 0.02201
    > 